In [99]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn import datasets
import scipy.signal
import scipy.cluster.vq as vq
import scipy.io.wavfile as wav
import python_speech_features as speech
from sklearn import mixture
from sklearn.mixture import GaussianMixture as gmm
import scipy.spatial.distance as dist
import math

In [137]:
def BIC(k, Sigs, Koefs, Lengths, penalty):
    P1 = Lengths[Koefs[k][0]]* np.log(np.linalg.det(np.cov(Sigs[Koefs[k][0]])))
    P2 = Lengths[Koefs[k][1]] * np.log(np.linalg.det(np.cov(Sigs[Koefs[k][1]])))
    P3 = (Lengths[Koefs[k][0]]+Lengths[Koefs[k][1]])*np.log(np.linalg.det(np.cov(Sigs[Koefs[k][0]], Sigs[Koefs[k][1]])))
    bic= 0.5*P1 + 0.5*P2-0.5*P3  + penalty*(2.5*(Lengths[Koefs[k][0]]+Lengths[Koefs[k][1]]))
    print(bic)
    return bic

In [138]:
def fit_samples(samples, num):
	gmix = mixture.GaussianMixture(n_components=num, covariance_type='full')
	gmix.fit(samples)
	return (gmix.means_) #ovo su centroide

#glavna funkcija:
def calculate(Means, Mfccs, Sigs, Lengths, Freqs, M, num, penalty):

    Koefs = []
    for i in range(0, num):
        Koefs.append([0,0])
    
    N = min(num, 5)
    MaxVals = np.zeros((N))
    
    #racunanje N najblizih koeficijenata
    for i in range(0,num-1):
        for j in range(2,num):
            if(j>i):
                
                for l in range(0,N):
                    if(M[i][j]>=MaxVals[l]):
                        for k in range(l+1, N):
                            MaxVals[k]=MaxVals[k-1]
                            Koefs[k] = Koefs[k-1]
                        MaxVals[l] = M[i][j]
                        Koefs[l] = [i,j]
                 
    #racunanje najveceg BIC
    Bics = []
    for i in range(0, N):
        Bics.append(0)
        
    for i in range(0, N):
        Bics[i] = BIC(i, Sigs, Koefs, Lengths, penalty)
    
    max_bic = Bics[0]
    max_i=0
    for i in range(1,N):
        if(Bics[i]>max_bic):
            max_bic = Bics[i]
            max_i = i
            
    if(max_bic<0):
        return (Means_new, Mfccs_new, Sigs_new, Lengths_new, Freqs_new, M_new, -1)
    
    [i,j] = Koefs[max_i]
    sig_new = 0.5*Sigs[i]+0.5*Sigs[j]
    means_new = 0.5*Means[i]+0.5*Means[j]
    
    Means_new = []
    Mfccs_new = []
    Sigs_new = []
    Lengths_new = []
    Freqs_new = []
    M_new = np.zeros((num, num))
    
    for k in range (0,num):
        if(k!=i and k!=j):
            Sigs_new.append(Sigs[k])
            Mfccs_new.append(Mfccs[k])
            Means_new.append(Means[k])
            Lengths_new.append(Mfccs[k].shape[0])
            Freqs_new.append(Freqs[k])
    
    for k in range(0, num-1):
        if(k<i):
            for l in range(0, num-1):
                if(l<i):
                    M_new[k][l]=M[k][l]
                else:
                    M_new[k][l]=M[k][l+1]
        else:
            for l in range(0, num-1):
                if(l<i):
                    M_new[k][l]=M[k+1][l]
                else:
                    M_new[k][l]=M[k+1][l+1]
                    
    for k in range(0, num-1):
        if(k<j):
            for l in range(0, num-1):
                if(l<j):
                    M_new[k][l]=M[k][l]
                else:
                    M_new[k][l]=M[k][l+1]
        else:
            for l in range(0, num-1):
                if(l<j):
                    M_new[k][l]=M[k+1][l]
                else:
                    M_new[k][l]=M[k+1][l+1]

    freq = means_new
    freq_sum = np.sum(freq)
    freq = (freq_sum - freq)/freq_sum
    
    for k in range(0, num-2):
        M_new[num-2][k] = np.abs(dist.cosine(Freqs[k], freq))
               
    Lengths_new.append(speech.mfcc(sig_new).shape[0])
    Means_new.append(means_new)        
    Mfccs_new.append(speech.mfcc(sig_new))
    Sigs_new.append(sig_new)
    Freqs_new.append(freq)
    return (Means_new, Mfccs_new, Sigs_new, Lengths_new, Freqs_new, M_new, 1)

In [147]:
(rate,sig1) = wav.read("richard3.wav")
(rate,sig2) = wav.read("amy3.wav")
(rate,sig3) = wav.read("derek3.wav")
(rate,sig4) = wav.read("paolo3.wav")
(rate,sig5) = wav.read("nilofer3.wav")

People =[sig1, sig2, sig3, sig4, sig5]
Sigs=[]

Clusters = []

k=1
for sig in People:
    while((2000*k)<sig.shape[0]):
        Sigs.append(sig[(k-1)*2000: k*2000])
        k = k+1
        
print(len(Sigs))
num = len(Sigs)
        
mfcc_feat = np.copy(speech.mfcc(Sigs[0]))
Lengths = []#mfcc lengths
Mfccs = []
Freqs = []
    
for i in range(0, num):
    mfcc = speech.mfcc(Sigs[i])
    Mfccs.append(mfcc)
    if(i>0):
         mfcc_feat = np.vstack((mfcc_feat, speech.mfcc(Sigs[i])))
    Lengths.append(mfcc.shape[0])   
    
Means = fit_samples(mfcc_feat, num)

for i in range(0, num):
    freq = Means[i]
    freq_sum = np.sum(freq)
    freq = (freq_sum - freq)/freq_sum
    Freqs.append(freq)
    
M = np.zeros((num,num))
for i in range(0,num):
    for j in range(0,num):
        if(i==j):
            M[i][j]=0
        else:
            M[i][j] = np.abs(dist.cosine(Freqs[i], Freqs[j]))
        

[[ 897  897]
 [ 402  402]
 [   2    2]
 ..., 
 [   7    7]
 [-498 -498]
 [-752 -752]]
[[ -418  -418]
 [  275   275]
 [  197   197]
 ..., 
 [-3704 -3704]
 [-2750 -2750]
 [-1549 -1549]]
[[ -516  -516]
 [  488   488]
 [ 1622  1622]
 ..., 
 [-4650 -4650]
 [-5068 -5068]
 [-5104 -5104]]
[[-5334 -5334]
 [-6021 -6021]
 [-6803 -6803]
 ..., 
 [  589   589]
 [  460   460]
 [  120   120]]
[[ -321  -321]
 [ -840  -840]
 [-1288 -1288]
 ..., 
 [ 5214  5214]
 [ 4944  4944]
 [ 4537  4537]]
[[3954 3954]
 [3175 3175]
 [2998 2998]
 ..., 
 [ 634  634]
 [ 665  665]
 [ 715  715]]
[[  721   721]
 [  582   582]
 [  461   461]
 ..., 
 [-4618 -4618]
 [-5053 -5053]
 [-5299 -5299]]
[[-5262 -5262]
 [-5289 -5289]
 [-5652 -5652]
 ..., 
 [ -912  -912]
 [ -170  -170]
 [ -197  -197]]
[[-1105 -1105]
 [-1394 -1394]
 [ -538  -538]
 ..., 
 [-1634 -1634]
 [-1250 -1250]
 [ -124  -124]]
[[-1042 -1042]
 [-1762 -1762]
 [  247   247]
 ..., 
 [   75    75]
 [ -169  -169]
 [ -389  -389]]
[[-591 -591]
 [-751 -751]
 [-807 -807]
 ...,

In [140]:
ind = 1
penalty = 0

while(ind>0 and num>1):
    (Means, Mfccs, Sigs, Lengths, Freqs, M, ind) = calculate(Means, Mfccs, Sigs, Lengths, Freqs, M, num, penalty)
    num = num-1
len(Sigs)

C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: divide by zero encountered in log
C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: invalid value encountered in double_scalars


nan
nan
nan


KeyboardInterrupt: 

In [145]:
k

354